Example from

https://marcotcr.github.io/lime/tutorials/Lime%20-%20basic%20usage%2C%20two%20class%20case.html


https://github.com/marcotcr/lime/tree/master/doc/notebooks

and *Deep Learning with Python*, Chapter 6.

Bug solved here: https://github.com/marcotcr/lime/issues/236


In [1]:
# ensure plots print without needing to call show()
%matplotlib inline

In [2]:
# import our packages
import pandas
import numpy
import os
import gzip
import sys
import pickle
import textwrap

import seaborn
import wvpy.util

import sklearn.metrics
from sklearn.pipeline import make_pipeline

In [3]:
# set up Keras imports, this can be brittle
# no longer import keras, import tensorflow.keras
import tensorflow.keras as keras
# https://github.com/keras-team/keras/issues/12379#issuecomment-473823330
from tensorflow.keras import *
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
from tensorflow.keras.layers import *
#from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
#from keras.models import Model
#from keras.optimizers import RMSprop
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/RMSprop
from tensorflow.keras.optimizers import *

In [4]:
# get our data and language model through data adaptors
sys.path.append('.')
# put data load on our path
sys.path.append('../data/IMDB')
from load_IMDB import load_IMDB


In [5]:
# load the IMDB data
train_data, test_data = load_IMDB()

In [6]:
(train_data.data[5], train_data.target[5])

("I would put this at the top of my list of films in the category of unwatchable trash! There are films that are bad, but the worst kind are the ones that are unwatchable but you are suppose to like them because they are supposed to be good for you! The sex sequences, so shocking in its day, couldn't even arouse a rabbit. The so called controversial politics is strictly high school sophomore amateur night Marxism. The film is self-consciously arty in the worst sense of the term. The photography is in a harsh grainy black and white. Some scenes are out of focus or taken from the wrong angle. Even the sound is bad! And some people call this art?<br /><br />",
 0)

In [7]:
(test_data.data[5], test_data.target[5])

("I had high hopes for this one until they changed the name to 'The Shepherd : Border Patrol, the lamest movie name ever, what was wrong with just 'The Shepherd'. This is a by the numbers action flick that tips its hat at many classic Van Damme films. There is a nice bit of action in a bar which reminded me of hard target and universal soldier but directed with no intensity or flair which is a shame. There is one great line about 'being p*ss drunk and carrying a rabbit' and some OK action scenes let down by the cheapness of it all. A lot of the times the dialogue doesn't match the characters mouth and the stunt men fall down dead a split second before even being shot. The end fight is one of the better Van Damme fights except the Director tries to go a bit too John Woo and fails also introducing flashbacks which no one really cares about just gets in the way of the action which is the whole point of a van Damme film.<br /><br />Not good, not bad, just average generic action.",
 0)

## Training a classifier

Let's tokenize the data.

In [9]:
vocab_size = 10000
max_len = 300
embed_dim = 24

tok = keras.preprocessing.text.Tokenizer(num_words=vocab_size)
# all fitting, including tokenizers is done only on training data
tok.fit_on_texts(train_data.data)

# shuffle rows of training data in case neural net batching takes in ordered chunks
permutation = numpy.random.choice(len(train_data.data), size=len(train_data.data), replace=False)
train_data_shuffled = [train_data.data[i] for i in permutation]
train_target_shuffled = [train_data.target[i] for i in permutation]

# convert training data
train_text = tok.texts_to_sequences(train_data_shuffled)
train_text = keras.preprocessing.sequence.pad_sequences(
    train_text, maxlen=max_len,
    padding="post", truncating="post")
train_target = numpy.asarray(train_target_shuffled, dtype=float)

# convert test data
test_text = tok.texts_to_sequences(test_data.data)
test_text = keras.preprocessing.sequence.pad_sequences(
    test_text, maxlen=max_len,
    padding="post", truncating="post")
test_target = numpy.asarray(test_data.target, dtype=float)

We now define our neural net.



In [ ]:
# define our neural net
model = Sequential()
model.add(Embedding(vocab_size, embed_dim, input_length = max_len))
model.add(Bidirectional(LSTM(64)))  # or model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Save the initial state of the model, this lets us re-train. Having to pick the epoch and re-trian in this way is a bit non-standard and is evidence we are running this example on too little data and/or haven't taken enough precautions against over-fit (such as drop-out layers). Usually more epochs is better.

In [ ]:
wts = model.get_weights()

In [ ]:
history = model.fit(train_text, train_target, epochs=20,
                    batch_size=256, validation_split=0.2)

In [ ]:
training_trajectory = pandas.DataFrame(history.history)
training_trajectory

In [ ]:
seaborn.lineplot(data = training_trajectory[['loss', 'val_loss']])

In [ ]:
best_epoch_id = training_trajectory.val_loss.idxmin()
best_epoch_id

The early divergence of the train and validation curves is a bad sign.

For this data set a small epochs is pretty good (using validation loss as the criterion).

This is a stong sign of not having enough data for the task of learning the language model/embedding.

We will re-run to an early stage and look at our results.

In [ ]:
model.set_weights(wts)
model.fit(train_text, train_target, epochs=best_epoch_id + 1,
                    batch_size=32, validation_split=0.2)

In [ ]:
# Tensorflow now implements .predict_proba() as .predict()
preds_train = pandas.DataFrame(model.predict(train_text))
preds_train.columns = ["prediction"]
preds_train["actual"] = train_target


In [ ]:
preds_test = pandas.DataFrame(model.predict(test_text))
preds_test.columns = ["prediction"]
preds_test["actual"] = test_target


In [ ]:
wvpy.util.plot_roc(
    preds_train["prediction"],
    preds_train["actual"],
    title='training performance')

In [ ]:
wvpy.util.threshold_plot(
    preds_train,
    pred_var="prediction",
    truth_var="actual",
    plotvars=('precision', 'recall', 'accuracy'),
    title='training performance',
)

In [ ]:
# estimate accuracy on train
numpy.mean((preds_train.actual >= 0.5) == (preds_train.prediction >= 0.5))


In [ ]:
wvpy.util.plot_roc(
    preds_test["prediction"],
    preds_test["actual"],
    title='test performance')

In [ ]:
wvpy.util.threshold_plot(
    preds_test,
    pred_var="prediction",
    truth_var="actual",
    plotvars=('precision', 'recall', 'accuracy'),
    title='test performance',
)

In [ ]:
# estimate accuracy on test
numpy.mean((preds_test.actual >= 0.5) == (preds_test.prediction >= 0.5))


In [ ]:
# accuracy at another threshold
numpy.mean((preds_test.actual >= 0.3) == (preds_test.prediction >= 0.3))